# Maria Sofia Alvarez Lopez

In [1]:
#ESAI
import time
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
plt.style.use('dark_background')

## Diferencias finitas

Derivada central:

$$\frac{df}{dx} \approx \frac {f(x_0+h)-f(x_0-h)} {2h} $$

Segunda derivada Central:

$$\frac{d^2f}{dx^2}\approx \frac {f(x_0+h)-2 f(x_0)+f(x_0-h)} {h^2}$$

### Ecuación de onda
Vamos a resolver la ecuación de onda en una dimensión:

$$ \frac{\partial^2 u}{\partial t^2} =
{c^2} \frac{\partial^2 u}{\partial x^2} $$

$$u = u(t,x)$$

En términos de diferencias finitas queda

$$\frac{u^{n+1}_{i}-2u^{n}_{i}+u^{n-1}_{i}}{(\Delta t)^2}  = c^2\frac{u^{n}_{i+1}-2u^{n}_{i}+u^{n}_{i-1}}{(\Delta x)^2}  $$

$$u^{n+1}_{i}-2u^{n}_{i}+u^{n-1}_{i} = r(u^{n}_{i+1}-2u^{n}_{i}+u^{n}_{i-1})$$

donde $r = (\Delta t)^2 \,c^2/(\Delta x)^2$. 

$$u^{n+1}_{i} = 2u^{n}_{i}-u^{n-1}_{i}+ r(u^{n}_{i+1}-2u^{n}_{i}+u^{n}_{i-1})$$

$$u^{n+1}_{i} = 2u^{n}_{i}-u^{n-1}_{i}+ ru^{n}_{i+1}-2ru^{n}_{i}+ru^{n}_{i-1}$$

$$u^{n+1}_{i} = 2u^{n}_{i}(1-r)-u^{n-1}_{i}+ ru^{n}_{i+1}+ru^{n}_{i-1}$$

$$u^{n+1}_{i} = au^{n}_{i}-u^{n-1}_{i}+ r(u^{n}_{i+1}+u^{n}_{i-1})$$

donde $a=2(1-r)$. Es necesario contar con condiciones de frontera ($u(0, t) = l_0, u(L,t) = l_L$ y condiciones iniciales para la posición ($u(0,x) = f(x)$) y la velocidad ($du(0,x)/dx = v_0(x)$)

para encontrar $u^{0-1}_i$ se puede hacer uso de definición de primera derivada numérica:

$$\frac{\partial u }{ \partial t} = v_0$$ 

$$\frac{u^{1}_{i}-u^{-1}_{i}}{2\Delta t} = v0$$

$$u^{-1}_{i} = u^{1}_{i} - 2v_0\Delta t$$

Al sustituir en la ecuación de onda discretizada para $t = 0$:

$$u^{1}_{i} = au^{0}_{i}-(u^{1}_{i} - 2v_0\Delta t)+ r(u^{0}_{i+1}+u^{0}_{i-1})$$

$$u^{1}_{i} = \frac{a}{2}u^{0}_{i}+ \frac{r}{2}(u^{0}_{i+1}+u^{0}_{i-1}) + v_0\Delta t$$

Para que la solución sea estable se requiere que $a$ sea mayor que cero, lo que implica

$$0< 2\left( 1- \frac{c^2(\Delta t)^2}{(\Delta x)^2}\right)$$
despejando $\Delta t$
$$\Delta t < \Delta x / c$$

**Ejemplo con extremos fijos:**



In [2]:
dx = 0.02
L = 1.0
t_max = 1.5
c = 1.8
# Condiciones de frontera
l0 = 0
lL = 0

def pos_ini(x):
    sig = 0.1
    return 1.0/np.sqrt(sig)*np.exp(-((x-0.3)**2)/sig**2)

def vel_ini(x):
    return np.zeros(len(x))

def ecuacion_onda_1D(dx, t_max, L, c, l0, lL, pos_ini, vel_ini):
    dt = dx/(1.5*c)
    r = (c*dt/dx)**2
    a = 2*(1-r)
    x = np.linspace(0, L, num=1+int(np.round(L/dx)))
    v0 = vel_ini(x)
    lt = 1+int(np.round(t_max/dt))
    lx = len(x)
    U = np.zeros([lt, lx])
    U[0,:] = pos_ini(x)
    U[:,0] = l0
    U[:,-1] = lL
    
    for i in range(1,lx-1):
        U[1, i] = 0.5*a*U[0,i] + 0.5*r*(U[0,i+1] + U[0,i-1]) + dt*v0[i]
        
    for n in range(1,lt-1):
        for i in range(1,lx-1):
            U[n+1,i] = a*U[n,i] -U[n-1,i] + r*(U[n,i+1] + U[n,i-1])
    return x,U

x, U = ecuacion_onda_1D(dx, t_max, L, c, l0, lL, pos_ini, vel_ini)
print(U.shape)

(204, 51)


In [3]:
def update(num_frame, U, line):
    line.set_ydata(U[num_frame, :])
    return line,

def crear_animacion(x, U, name):
    N = U.shape[0]
    fig = plt.figure(figsize=(9,5), dpi=75)
    ax = plt.gca()
    ax.set_xlim([0,L])
    ax.set_ylim([np.min(U)-1, np.max(U)+1])
    line, = ax.plot(x,U[0],'r')
    plt.tight_layout()
    ani = animation.FuncAnimation(fig, update, N, fargs=(U,line))
    ani.save(name,fps=30)
    plt.close(fig)
    
crear_animacion(x, U, 'ecuacion_onda_1D.mp4')

In [4]:
# Mostrar video
from IPython.display import HTML
from base64 import b64encode

name = 'ecuacion_onda_1D.mp4'
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=675 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

## Condiciones para extremos libres

En este caso los extremos no permanecen en un valor fijo, se actualizan para cada instante de tiempo así:

$$u(t+\Delta t, 0) = u(t+\Delta t, \Delta x)$$

$$u(t+\Delta t, l_L) = u(t+\Delta t, l_L-\Delta x)$$

Es decir.

$$u^{n+1}_0 = u^{n+1}_1$$

$$u^{n+1}_{l_L} = u^{n+1}_{l_L-1}$$


In [5]:
dx = 0.05
L = 1.0
t_max = 100
c = 1.0/32

def pos_ini(x):
    sig = 0.1
    return 1.0/np.sqrt(sig)*np.exp(-((x-0.5)**2)/sig**2)

def vel_ini(x):
    return np.zeros(len(x))

def ecuacion_onda_1D_extremos_libres(dx, t_max, L, c, pos_ini, vel_ini):
    dt = dx/(180*c)
    r = (c*dt/dx)**2
    a = 2*(1.0-r)
    x = np.linspace(0, L, num=1+int(np.round(L/dx)))
    v0 = vel_ini(x)
    lt = 1+int(np.round(t_max/dt))
    lx = len(x)
    U = np.zeros([lt, lx])
    U[0,:] = pos_ini(x)
    for i in range(1,lx-1):
        U[1, i] = 0.5*a*U[0,i] + 0.5*r*(U[0,i+1] + U[0,i-1]) + dt*v0[i]
    for n in range(1,lt-1):
        for i in range(1,lx-1):
            U[n+1,i] = a*U[n,i] -U[n-1,i] + r*(U[n,i+1] + U[n,i-1])
        U[n+1,0] = U[n+1,1]
        U[n+1,-1] = U[n+1,-2]
    return x,U

x, U = ecuacion_onda_1D_extremos_libres(dx, t_max, L, c, pos_ini, vel_ini)
print(U.shape)
U = U[::50]
print(U.shape)

(11251, 21)
(226, 21)


In [6]:
def update(num_frame, x, U, line, ax):
    y = U[num_frame, :]
    line.set_ydata(y)
    ax.collections.clear()
    piso = np.ones(len(x))*(-7)
    ax.fill_between(x,y,piso,piso<y, color='dodgerblue')
    return line,

def crear_animacion(x, U, name):
    N = U.shape[0]
    fig = plt.figure(figsize=(9,5), dpi=75)
    ax = plt.gca()
    ax.set_xlim([0,L])
    ax.set_ylim([-7,7])
    line, = ax.plot(x,U[0],'r')
    plt.tight_layout()
    ani = animation.FuncAnimation(fig, update, N, fargs=(x, U, line, ax))
    ani.save(name,fps=30)
    plt.close(fig)
    
crear_animacion(x, U, 'ecuacion_onda_1D_extremos_libres2.mp4')

In [7]:
# Mostrar video
from IPython.display import HTML
from base64 import b64encode

name = 'ecuacion_onda_1D_extremos_libres2.mp4'
mp4 = open(name,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=675 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)